# **任务10 循环神经网络 - 文本分类 | RNN - Text Classification**

本质上还是分类任务，只是将提取特征的方法换成了更适合处理文本的循环神经网络。

___


## 1. **新的层结构**

### 1.1 **torch循环神经网络层（及其变体）**
循环神经网络是一种专门用来处理时序数据的结构，它能很有效的关注数据的先后关系。

### **使用方法**

它同样来自于 `torch` 的 `nn` 子类中

**循环神经网络（RNN）**：`RNN = nn.RNN(embedding_dim, hidden_size, num_layers, batch_first=True)`

- RNN 是为处理序列数据（如文本、时间序列）设计的神经网络，核心是通过「隐藏状态（Hidden State）」传递上下文信息：每一步的输出不仅依赖当前输入，还依赖上一步的隐藏状态，实现对序列时序关系的建模。

**长短期记忆网络（LSTM）**：`LSTM = nn.LSTM(embedding_dim, hidden_size, num_layers, batch_first=True)`

- LSTM 是 RNN 的变体，核心目标是解决 RNN 的长序列梯度消失 / 爆炸问题，实现对长依赖的有效捕捉。

**门控单元循环神经网络（GRU）**：`GRU = nn.GRU(embedding_dim, hidden_size, num_layers, batch_first=True)`

- GRU 是 LSTM 的简化版，核心目标是解决 LSTM 结构复杂、计算效率低的问题，在保证近似 LSTM 性能的前提下，降低模型复杂度和计算成本。

它们的结构很相似，所以可以完全替换。

对输入数据的维度要求是 [批次大小, 序列长度, 特征向量维度]，其中特征向量表示的是某个字或符号的特征值，通过一种索引映射关系构建它们。

早期特征向量是对应字符的**独热编码**在先前的任务4中有提到，由于独热编码的特点，向量维度需要与词汇表长度相同，当词汇过多时可能导致计算量过大，同时这种各维度垂直的向量关系不能表示近义词，所以不算高效。

`embedding_dim` 为特征向量维度, `hidden_size` 隐藏层维度，会改变输出的特征向量维度, `num_layers` 层数，决定模型复杂度

循环神经网络会有两个输出，一个是**核心输出**，一个是**隐藏输出**，隐藏输出的作用是作为不同细胞间的信息交流，由于最终输出没有下一个细胞了，所以这个输出没有去向，但依然可以作为数据在后续计算中应用。

In [85]:
import torch
import torch.nn as nn

batch_size = 16
seq_num = 10
embedding_dim, hidden_size, num_layers = 16, 32, 2

input_tensor = torch.rand((batch_size, seq_num, embedding_dim))

# batch_first表示第一个维度是批次维度
rnn = nn.RNN(embedding_dim, hidden_size, num_layers, batch_first=True)
lstm = nn.LSTM(embedding_dim, hidden_size, num_layers, batch_first=True)
gru = nn.GRU(embedding_dim, hidden_size, num_layers, batch_first=True)

output_rnn, _ = rnn(input_tensor)
output_lstm, _ = lstm(input_tensor)
output_gru, _ = gru(input_tensor)

print('输入张量：', input_tensor.shape)
print('三种循环网络的输出：')
print(output_rnn.shape)
print(output_lstm.shape)
print(output_gru.shape)

输入张量： torch.Size([16, 10, 16])
三种循环网络的输出：
torch.Size([16, 10, 32])
torch.Size([16, 10, 32])
torch.Size([16, 10, 32])


### 1.2 **torch嵌入层**

先前我们说可以用独热编码作为字符的特征向量，但也提到了它的劣势。嵌入层是解决其的有效方案，通过让模型在不断学习中自动更改特征向量的数值大小，更有利于模型理解不同词的使用关系，比如近义词的向量的空间距离可能更小，用这种方式来理解词义，同时不会受词汇表大小的影响。

它同样来自于 `torch` 的 `nn` 子类中 `nn.Embedding(vocab_size, embedding_dim)`

`vocab_size` 是词汇表大小，`embedding_dim` 是特征向量维度

虽然嵌入层不受限于词汇表大小，但仍然需要vocab_size参数用于索引构建和校验

由于嵌入层是根据字符索引构建特征向量的，所以输入有所不同，必须是整型张量。

In [86]:
import random

embedding_dim = 16
batch_size = 2

vocab_dict = {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4}
vocab_size = len(vocab_dict)
print(f'词汇表：{vocab_dict}')

batch_list = []
for _ in range(batch_size):
    # 随机生成一个长度为seq_num的句子
    sentence = random.choices(list(vocab_dict.keys()), k=seq_num)
    print(f'- 句子{_}：{sentence}')
    # 转换成索引
    sentence_index = [vocab_dict[char] for char in sentence]
    print(f'- 句子{_}的索引：{sentence_index}')
    # 转换成张量
    sentence_tensor = torch.tensor(sentence_index)
    # 添加批次维度
    sentence_tensor = sentence_tensor.unsqueeze(dim=0)
    print(f'- 句子{_}的张量：{sentence_tensor}')
    batch_list.append(sentence_tensor)
# 拼接成一整个批次
input_tensor = torch.cat(batch_list, dim=0)

embedding = nn.Embedding(vocab_size, embedding_dim)

output_tensor = embedding(input_tensor)
print(f'输入张量：{input_tensor.shape}')
print(f'输出张量：{output_tensor.shape}')

词汇表：{'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4}
- 句子0：['a', 'c', 'c', 'c', 'e', 'a', 'e', 'b', 'a', 'e']
- 句子0的索引：[0, 2, 2, 2, 4, 0, 4, 1, 0, 4]
- 句子0的张量：tensor([[0, 2, 2, 2, 4, 0, 4, 1, 0, 4]])
- 句子1：['b', 'b', 'a', 'b', 'd', 'a', 'b', 'a', 'd', 'b']
- 句子1的索引：[1, 1, 0, 1, 3, 0, 1, 0, 3, 1]
- 句子1的张量：tensor([[1, 1, 0, 1, 3, 0, 1, 0, 3, 1]])
输入张量：torch.Size([2, 10])
输出张量：torch.Size([2, 10, 16])


## 2. **定义模型**

In [87]:
class Model(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers, output_size):
        super(Model, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.embedding(x)
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])  # 取最后一个时间步的输出

        return out

## 3. **数据生成**

对于运动或食物的描述来判断喜欢或不喜欢。

由于需要将句子分离成词汇或字符，这里我用了 `thulac` 库用于中文分词。

这里直接使用机器学习库 `sklearn` 库的 `train_test_split` 方法来分割数据集了

In [88]:
import thulac
from sklearn.model_selection import train_test_split

thu1 = thulac.thulac()

food_list = ['苹果', '饼干', '汉堡', '披萨', '饺子', '面条', '香菇', '香菜', '大蒜', '馄饨', '面包', '煎饼', '面皮', '炒饭', '炒面', '蛋糕', '榴莲', '菠萝', '香蕉']
play_list = ['足球', '篮球', '羽毛球', '网球', '棒球', '游泳', '橄榄球', '跳绳', '跑步', '滑冰', '滑雪', '曲棍球', '冰球', '射箭', '台球', '马拉松']

food_like = [
    "我喜欢吃……，它很甜很好吃。",
    "……很香，我喜欢吃。",
    '我觉得……很好吃。',
    '我喜欢吃……，每次吃都会觉得美味。',
    '我非常喜欢……，它非常美味。',
    ' 是我最喜欢的食物。',
    '我特别喜欢吃……。',
    '我抵抗不了……的诱惑。',
    '……是我最喜欢的食物。',
    '我真的特别喜欢吃……。',
    '你不觉得……非常好吃吗？',
    '我觉得……是世界上最好吃的。',
    '我超爱……，吃一口就幸福感爆棚。',
    '我对……情有独钟，那滋味太赞了。',
    '每次品尝……，都感觉味蕾在欢呼。',
    '我痴迷于……，它的美味无可抵挡。',
    '一提到……我就流口水，真的太喜欢了。',
    '……是我的本命食物，怎么吃都不腻。',
    '我热衷于……，它是我生活中的小确幸美食。',
    '……能瞬间点亮我的心情，太爱这一口了。',
    '我简直为……疯狂，美味到犯规。',
    '……给我带来超多快乐，必须喜欢啊。',
    '我被……的美味折服，时不时就想来点儿。',
    '只要有……，这顿饭就超满足。',
]

food_unlike = [
    '我讨厌……，我受不了这个味道。',
    '我不喜欢……，它的味道让我恶心。',
    '……那么难吃。',
    '我吃不下……。',
    '……太难吃了，我吃不下去。',
    '我受不了……的味道。',
    '不要……，它太难吃了。',
    '……太难闻了，我不要吃。',
    '我太讨厌……的味道了。',
    '我极其厌恶……，闻到味儿就难受。',
    '……的味道让我作呕，实在难以下咽。',
    '我一看到……就没胃口，真心不喜欢。',
    '……真的很倒胃口，我绝对不会碰。',
    '我嫌弃……，它的口感太差劲了。',
    '……对我来说就是噩梦，味道太恐怖。',
    '别给我……，那股味儿我受不了。',
    '我避开……还来不及呢，太难吃了。',
    '……让我敬而远之，味道实在不敢恭维。',
    '我和……绝缘，那味道我无法忍受。',
    '一想到……的味道，我就头皮发麻。',
    '……是我的“黑名单”食物，绝不吃它。',
    '我对……毫无好感，味道太糟糕。',
    '……真不是我的菜，难吃程度五颗星。',
    '我见到……就想绕道走，太难闻了。',
    '我觉得……难闻。'
]

play_like = [
    '我喜欢……，他让我觉得放松。',
    '我很喜欢……，尽管我不擅长它。',
    '我擅长……，所以……是我最喜欢的运动。',
    '……非常有趣，所以我每天都去。',
    '我每天都去……来放松自己。',
    '……让我觉得很有动力去运动。',
    '……是我最喜欢的运动。',
    '我超享受……，玩起来特别解压。',
    '……总能让我忘却烦恼，超爱这项运动。',
    '我热衷于……，每次参与都活力满满。',
    '……是我的快乐源泉，一玩就停不下来。',
    '我超迷……，它让我变得更有活力。',
    '只要有空，我就去……，太好玩了。',
    '……给我带来无限乐趣，必须列为最爱。',
    '我对……上瘾，感觉越玩越带劲。',
    '我超爱投身于……，那种畅快难以言表。',
    '……让我找到了激情，玩不够啊。',
    '每次……都让我热血沸腾，超喜欢。',
    '我钟情于……，它让生活变得更精彩。',
    '……能让我尽情释放能量，超赞的运动。',
    '我一玩……就兴奋，它是我的心头好运动。'
]

play_unlike = [
    '我不擅长……，它让我很累。',
    '我不喜欢……，我觉得它很无聊。',
    '……真的很没意思。',
    '……太难了，不适合我。',
    '我认为……很无聊，我不喜欢。',
    '我不喜欢……。',
    '……实在是有些无聊了。',
    '我一玩……就犯困，实在提不起兴趣。',
    '我觉得……枯燥乏味，完全不想尝试。',
    '……对我来说就是折磨，毫无乐趣可言。',
    '我尝试过……，但真心觉得无聊透顶。',
    '我受不了……的单调，玩几次就放弃了。',
    '……让我感到无趣至极，不会再碰。',
    '我对……无感，找不到一点好玩的地方。',
    '……太沉闷了，我宁愿闲着也不玩。',
    '我不理解……的乐趣所在，就是不喜欢。',
    '我避开……，它无法给我带来任何愉悦。',
    '一想到……，我就觉得没意思，不想动。',
    '……是我最不想参与的，太没劲了。',
    '……实在引不起我的兴致，太乏味了。',
    '我和……气场不合，玩起来别扭'
    '我对……无感。',
    "我对……无感，它很枯燥。",
]

X_data = []
# Y表示喜欢和不喜欢的二分类
Y_data = []
for like_food in food_like:
    for food in food_list:
        food = like_food.replace('……', food)
        result = [i[0] for i in thu1.cut(food)]
        X_data.append(result)
        Y_data.append(0)
for like_play in play_like:
    for play in play_list:
        play = like_play.replace('……', play)
        result = [i[0] for i in thu1.cut(play)]
        X_data.append(result)
        Y_data.append(0)

for unlike_food in food_unlike:
    for food in food_list:
        food = unlike_food.replace('……', food)
        result = [i[0] for i in thu1.cut(food)]
        X_data.append(result)
        Y_data.append(1)

for unlike_play in play_unlike:
    for play in play_list:
        play = unlike_play.replace('……', play)
        result = [i[0] for i in thu1.cut(play)]
        X_data.append(result)
        Y_data.append(1)

# 创建词汇表
vocab = set()
for sentence in X_data:
    vocab.update(sentence)
vocab = sorted(list(vocab))
word_to_idx = {word: idx for idx, word in enumerate(vocab)}

max_len = max(len(sentence) for sentence in X_data)
print("最大长度", max_len)
# 将文本数据转换为索引表示的张量
X_data_idx = []
for sentence in X_data:
    sentence_idx = [word_to_idx[word] if word in word_to_idx else len(word_to_idx) for word in sentence]
    if len(sentence_idx) < max_len:
        sentence_idx += [len(word_to_idx)] * (max_len - len(sentence_idx))
    X_data_idx.append(sentence_idx)

X_data = torch.tensor(X_data_idx, dtype=torch.long)
Y_data = torch.tensor(Y_data, dtype=torch.long)

# 训练数据
img_size = 64
batch_size = 128
train_x, val_x, train_y, val_y  = train_test_split(X_data, Y_data, test_size=0.1)

# 划分批次
def split_batch(data, batch_size):
    # 核心操作：沿第一个维度（dim=0）分割，保留后续所有维度
    split_tensors = torch.split(data, batch_size, dim=0)
    # 转为列表返回（torch.split返回tuple，列表更易操作）
    return list(split_tensors)

train_x_batch = split_batch(train_x, batch_size)
train_y_batch = split_batch(train_y, batch_size)

print('输入数据形状:', train_x.shape)
print('输入批次数量:', len(train_x_batch), '\t批次形状:', train_x_batch[0].shape)
print('标签数据形状:', train_y.shape)
print('输入批次数量:', len(train_y_batch), '\t批次形状:', train_y_batch[0].shape)

Model loaded succeed
最大长度 18
输入数据形状: torch.Size([1457, 18])
输入批次数量: 12 	批次形状: torch.Size([128, 18])
标签数据形状: torch.Size([1457])
输入批次数量: 12 	批次形状: torch.Size([128])


## 4. 模型训练

### 4.1 实例化模型、损失函数、优化器

任务是分类问题，使用交叉熵损失 `nn.CrossEntropyLoss()`。

In [89]:
vocab_size = len(word_to_idx) + 1  # 词汇表大小，加1是为了处理未登录词的索引
embedding_dim = 64
hidden_size = 64
num_layers = 2
output_size = 2

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Model(vocab_size, embedding_dim, hidden_size, num_layers, output_size).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

### 4.2 **迭代训练**

In [90]:
epochs = 80
for epoch in range(epochs):
    loss = None
    model.train()
    for i in range(len(train_x_batch)):
        x = train_x_batch[i].to(device)
        y = train_y_batch[i].to(device)

        # 前向传播，得到预测值
        output = model(x)
        # 计算损失
        loss = criterion(output, y)
        # 梯度清零，因为在每次反向传播前都要清除之前累积的梯度
        optimizer.zero_grad()
        # 反向传播，计算梯度
        loss.backward()
        # 更新权重和偏置
        optimizer.step()

    val_x = val_x.to(device)
    val_y = val_y.to(device)
    model.eval()
    output = model(val_x)
    val_loss = criterion(output, val_y)

    # 更改验证逻辑为适合分类任务的准确率和召回率
    if (epoch + 1) % 20 == 0:
        print(f'[epoch {epoch+1}]loss:', loss.item())
        print(f'\t val loss:', val_loss.item())


[epoch 20]loss: 0.00259727262891829
	 val loss: 0.08127949386835098
[epoch 40]loss: 0.0005537125398404896
	 val loss: 0.09274356067180634
[epoch 60]loss: 0.000258656800724566
	 val loss: 0.09987889975309372
[epoch 80]loss: 0.00015259011706802994
	 val loss: 0.10504584014415741


### 4.3 **测试模型**

使用 `model.eval()` 将模型改为测试模式，避免自动的梯度计算增加额外的计算量。

使用 `torch.argmax()` 将概率得分向量转换为类别索引。

In [91]:
test_text_list = [
    "我喜欢吃苹果，它很甜很好吃。",
    "香菜太难闻了，我不喜欢吃。",
    "香蕉很香，我喜欢吃。",
    "我觉得榴莲太难闻了，我不喜欢吃。",
    "我特别喜欢棒球，它太好玩了。",
    "我对羽毛球无感，它很枯燥。",
]
model.eval()
for test_text in test_text_list:
    test_result = [i[0] for i in thu1.cut(test_text)]
    test_data_idx = [word_to_idx[word] if word in word_to_idx else len(word_to_idx) for word in test_result]
    if len(test_data_idx) < max_len:
        test_data_idx += [len(word_to_idx)] * (max_len - len(test_data_idx))
    test_data = torch.tensor(test_data_idx, dtype=torch.long)

    input_tensor = torch.stack([test_data]).to(device)
    output = model(input_tensor)
    _, predicted = torch.max(output.data, 1)
    print(f"{test_text}: {'喜欢' if predicted.item() == 0 else '不喜欢'}")

我喜欢吃苹果，它很甜很好吃。: 喜欢
香菜太难闻了，我不喜欢吃。: 不喜欢
香蕉很香，我喜欢吃。: 喜欢
我觉得榴莲太难闻了，我不喜欢吃。: 不喜欢
我特别喜欢棒球，它太好玩了。: 喜欢
我对羽毛球无感，它很枯燥。: 不喜欢


## 5. **总结**

这是第一个文本任务，引入了循环神经网络和嵌入层。嵌入层的作用是将离散数据转换为可连续计算的数据，这种思想可以发挥很大的想象力。